# LIC-Analyzer

Dieses Notebook dient dazu die Lizenzen für **PLATO e1ns.methods** auszuwerten. Es wird dazu ein Zweig des SVN-Repositories **PLX_methods_worksheets** analysiert - Standard ist ``trunk`` - und ein Datensatz erstellt, der alle in den Lizenz-Dateien und Formblättern verwendeten Lizenz-Kategorien enthält.

## Lizenz-Dateien analysieren

Aus den Lizenz-Dateien werden unterschiedliche Daten extrahiert, die in Relation zum Kunden gespeichert werden. Der Kunde wird dabei dem Parameter ``company`` entnommen, der in der Lizenz-Datei im Abschnitt ``main`` aufgeführt ist. Die extrahierten Daten sind die Laufzeit der Lizenz (``expire``), der Zeitpunkt des Signierens (``modified``), der Pfad der Datei und die enthaltenen Methoden mit der Anzahl der verfügbaren Slots (``slotCount``), sowie der Anzahl an gleichzeitigen Nutzern (``floatingUserCount``).

## Formblätter analysieren

Alle Formblätter werden nach den Parametern ``__lic_category`` und ``__lic_is_catalog`` durchsucht. Die Formblätter (Name und Pfad zur Datei) werden in relation zur Lizenz-Kategorie gestellt. ``__lic_is_catalog`` dient später nur zum Filtern, wenn man ausschließlich Formblätter betrachten möchte, die keine Kataloge sind. Durch die gewählte relation lassen sich die Formblätter den Kunden zuordnen.

## DataFrame erstellen

Die zuvor ersatellten Datensätze werden zu einem [DataFrame](https://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) fusioniert um mit Hilfe von [pandas](https://pandas.pydata.org/) eine Analyse des Datensatzes durchführen zu können.

### Excel-Export

Ein DataFrame lässt sich in unterschiedliche Datei-Formate abspeichern. Für die Übernahme in e1ns sollte das Excel-Format verwendet werden.

## Verwendete Importe

In [ ]:
import os
import re

from collections import defaultdict
from datetime import datetime

from pandas import DataFrame
import pandas as pd

## Code zur Analyse der Lizenz-Dateien

In [ ]:
# Methoden ohne den Parameter 'slotCount'
METHODS_WITHOUT_SLOTS = ('architect',)
# Methoden ohne den Parameter 'floatingUserCount'
METHODS_WITHOUT_USERS = ()

def _parse_lic_metadata(file_):
    """Durchsucht die übergebene Lizenz-Datei nach den Metadaten 'company', 'expire' und 'modified'.
    """
    pattern_modified = re.compile('^\s*modified\s*=\s*(.+)$')
    find_modified = True
    pattern_kunde = re.compile('^\s*company\s*=\s*(.+)$')
    find_kunde = True
    pattern_expire = re.compile('^\s*expire\s*=\s*(\d{4}-\d{2}-\d{2})$')
    find_expire = True
    
    kunde = expire = modified = ''
    
    for i, line in enumerate(open(file_)):
        if find_modified:
            for match in re.finditer(pattern_modified, line):
                modified =  match.groups()[0]
                find_modified = False
                continue
        if find_kunde:
            for match in re.finditer(pattern_kunde, line):
                kunde =  match.groups()[0]
                find_kunde = False
                continue
        if find_expire:
            for match in re.finditer(pattern_expire, line):
                expire =  match.groups()[0]
                find_expire = False
                continue
        if not find_modified and not find_kunde and not find_expire:
            find_modified = find_kunde = find_expire = True
            break
            
    return kunde, expire, modified

def find_licfiles(directory=r'C:\Users\entrup\workspace\PLX_methods_worksheets\trunk\__documents__\licfiles'):
    """Erstellt ein verschateltes Dictionary mit der Struktur Kunden > Licfiles (Dateiname ohne Endung).
    
    Zu jedem Licfile wird der vollständige Pfad als 'path' gespeichert.
    """    
    licfiles_dict = defaultdict(list)
    
    for root, dirs, files in os.walk(directory):
        for file_ in files:
            name, ext = os.path.splitext(file_)
            if ext == '.lic':
                file_ = os.path.join(root, file_)
                kunde, expire, modified = _parse_lic_metadata(file_)
                licfile = {
                    'name': name,
                    'path': file_,
                    'expire': expire,
                    'modified': modified
                }     
                licfiles_dict[kunde].append(licfile)
    return licfiles_dict


def _analyze_licfile(licfile_dict):
    """Extrahiert die lizensierten Methoden aus der im Dictionary vermerkten Lizenzdatei.
    
    Das Dictionary wird genutzt, um die Anzahl an Slots und Benutzern pro gefunderer Methode zu speichern.
    Zusätzlich wird die Anzahl der gefundenen Methoden zurückgegeben.
    """
    pattern_cat = re.compile('^\s*\[method_([\w\d_]+)\]')
    pattern_users = re.compile('^\s*floatingUserCount\s*=\s*(\d+)')
    pattern_slots = re.compile('^\s*slotCount\s*=\s*(\d+)')
    find_method = True
    found_method = ''
    find_users = True
    find_slots = True
    methods_list = []
    method_dict = {}
    for i, line in enumerate(open(licfile_dict['path'])):
        if find_method:
            for match in re.finditer(pattern_cat, line):
                found_method = match.groups()[0]
                if found_method in METHODS_WITHOUT_SLOTS:
                    find_slots = False
                if found_method in METHODS_WITHOUT_USERS:
                    find_users = False
                method_dict['name'] = found_method
                find_method = False
        else:
            if find_users:
                for match in re.finditer(pattern_users, line):
                    found_user_count = match.groups()[0]
                    method_dict['users'] = int(found_user_count)
                    find_users = False
            if find_slots:
                for match in re.finditer(pattern_slots, line):
                    found_slot_count = match.groups()[0]
                    method_dict['slots'] = int(found_slot_count)
                    find_slots = False
            if not find_users and not find_slots:
                find_users = True
                find_slots = True
                find_method = True
                methods_list.append(method_dict)
                method_dict = {}
    licfile_dict['methods'] = methods_list
            

def analyze_licfiles(licfiles_dict):
    """Iteriert über alle im Dictionary hinterlegten Lizenz-Dateien und extrahiert die dort lizensierten Methoden.
    """
    for licfiles in licfiles_dict.values():
        for licfile in licfiles:
            _analyze_licfile(licfile)

## Code zur Analyse der Formblätter

In [ ]:
def find_used_lics(directory=r'C:\Users\entrup\workspace\PLX_methods_worksheets\trunk\__productstatus__'):
    """Iteriert über alle Formblätter im übergebenen Verzeichnis und liest die vergebenen Lizenz-Kategorien aus.
    """
    pattern_id = re.compile('^\s*WORKSHEET_ID\s*=\s*["\']([\w\d_]+)["\']')
    pattern_lic = re.compile('^\s*__lic_category\s*=\s*["\']([\w\d_]+)["\']')
    pattern_catalog = re.compile('^\s*__lic_is_catalog\s*=\s*(True|False)')
    used_lics = {}
    
    for root, dirs, files in os.walk(directory):
        for file_ in files:
            file_name, ext = os.path.splitext(file_)
            if ext == '.py':
                # print file_name
                full_path = os.path.join(root, file_)
                find_id = find_lic = find_catalog = True
                ws_id = lic_cat = ''
                is_catalog = False
                for i, line in enumerate(open(full_path)):
                    if find_id:
                        for match in re.finditer(pattern_id, line):
                            ws_id = match.groups()[0]
                            find_id = False
                    
                    if find_lic:
                        for match in re.finditer(pattern_lic, line):
                            lic_cat = match.groups()[0]  
                            # print lic_cat
                            find_lic = False

                    if find_catalog:
                        for match in re.finditer(pattern_catalog, line):
                            is_catalog = (match.groups()[0] == 'True')
                            # print is_catalog
                            find_catalog = False
                        
                    if not find_lic and not find_catalog and not find_id:
                        break
                
                if lic_cat:
                    used_in = used_lics.get(lic_cat, {'formsheets': []})
                    worksheet = {
                        'id': ws_id,
                        'name': file_name,
                        'path': full_path,
                        'is_catalog': is_catalog,
                    }
                    used_in['formsheets'].append(worksheet)
                    used_lics[lic_cat] = used_in
    return used_lics

## Datensatz generieren (trunk)

### Lizenz-Dateien

In [ ]:
kunden_licfiles = find_licfiles()
analyze_licfiles(kunden_licfiles)
print 'Anzahl Lizenz-Dateien: {}'.format(len(kunden_licfiles))
print
print 'Liste der Kunden mit e1ns-Lizenzen:'
print
print '\n'.join(sorted(kunden_licfiles.keys()))

### Formblätter

In [ ]:
used_lics = find_used_lics()
print 'Anzahl genutzter Lizenzen: {}'.format(len(used_lics))

In [ ]:
for key in used_lics.keys():
    used_lics[key]['formsheet_count'] = len([True for item in used_lics[key]['formsheets'] if not item['is_catalog']])
    used_lics[key]['catalog_count'] = len(used_lics[key]['formsheets']) - used_lics[key]['formsheet_count']

Liste mit allen im trunk genutzten Lizenz-Kategorien und für die Verwendung als Lic-Datei formatieren:

In [ ]:
lic_all = find_used_lics(directory=r'C:\Users\entrup\workspace\PLX_methods_worksheets\trunk')
print 'Anzahl genutzter Lizenzen: {}'.format(len(lic_all))

count_str = """
scioDbGuidlist = []
floatingUserCount = 100
slotCount = 100\n
"""
with open(r'C:\Users\entrup\Documents\Jupyter-Output\lic_all.txt', 'w') as f:
    for lic in sorted(lic_all, key=lambda v: v.lower()):
        f.write('[method_{}]{}'.format(lic, count_str))

## Datensatz nach pandas konvertieren

### Lizenz-Dateien

Es werden beide zuvor erzeugten Datensätze kombiniert, damit der Datensatz auch die vom jeweiligen Kunden verwendbaren Formblätter enthält.

In [ ]:
cols_kunden = [
    'Kunde',
    'Lizenzdatei',
    'Signiert',
    'Laufzeit',
    'Methode',
    'Anzahl Slots',
    'Anzahl Benutzer',
    'Formblatt',
    'ist Katalog?'
]

cols_methods = [
    'Methode',
    u'Anzahl Formblätter',
    'Anzahl Kataloge',
    'Formblatt',
    'ist Katalog?'
]

In [ ]:
def convert_kunden_dict_to_dataframe(kunden_dict, methods_dict):
    expanded_list = list()
    
    ws_without_id = set()

    for kunde, licfiles in kunden_dict.iteritems():
        for licfile in licfiles:
            for method in licfile['methods']:
                if method['name'] in methods_dict.keys():
                    for formsheet in methods_dict[method['name']]['formsheets']:
                        formblatt = formsheet['id']
                        if not formblatt:
                            formblatt = formsheet['name']
                            ws_without_id.add(formsheet['path'])
                        line = {
                            'Kunde': kunde,
                            'Lizenzdatei': licfile['name'],
                            'Signiert': datetime.strptime(licfile['modified'], '%a %b %d %H:%M:%S %Y') if licfile['modified'].strip() else '',
                            'Laufzeit': datetime.strptime(licfile['expire'], '%Y-%m-%d') if licfile['expire'].strip() else '',
                            'Methode': method['name'],
                            'Anzahl Slots':  method['slots'],
                            'Anzahl Benutzer':  method['users'],
                            'Formblatt': formblatt,
                            'ist Katalog?': formsheet['is_catalog']
                        }
                        expanded_list.append(line)
    
    df = DataFrame(expanded_list)
    df = df[cols_kunden]
    df.sort_values(by=['Kunde', 'Lizenzdatei', 'Methode', 'Formblatt'], inplace=True)
    
    # Methoden mit 'floatingUserCount == 0' sind nicht von Interesse:
    df_filtered = df[df['Anzahl Benutzer'] > 0]
    
    for ws in ws_without_id:
        print('[Debug] {} besitzt keine WORKSHEET_ID.'.format(ws))
    
    return df_filtered

In [ ]:
df_kunden = convert_kunden_dict_to_dataframe(kunden_licfiles, used_lics)

In [ ]:
print 'Größe des Datensatzen: {}'.format(df_kunden.shape)
df_kunden.head()

### Multiindex erzeugen, um Struktur des e1ns-Formblattes zu erhalten:

In [ ]:
cols2join_kunden = cols_kunden[0:-1]
df_kunden_multiindex = df_kunden.set_index(cols2join_kunden)

In [ ]:
df_kunden_multiindex.head()

### DataFrame mit den verfügbaren Methoden erstellen

In [ ]:
def convert_methods_dict_to_dataframe(methods_dict):
    expanded_list = list()

    for method, content in methods_dict.iteritems():
        for formsheet in content['formsheets']:
            formblatt = formsheet['id']
            if not formblatt:
                formblatt = formsheet['name']
            line = {
                'Methode': method,
                u'Anzahl Formblätter': content['formsheet_count'],
                'Anzahl Kataloge': content['catalog_count'],
                'Formblatt': formblatt,
                'ist Katalog?': formsheet['is_catalog']
            }
            expanded_list.append(line)
    
    df = DataFrame(expanded_list)
    df = df[cols_methods]
    df.sort_values(by=['Methode', 'Formblatt'], inplace=True)
    
    return df

In [ ]:
df_methods = convert_methods_dict_to_dataframe(used_lics)

In [ ]:
print 'Größe des Datensatzen: {}'.format(df_methods.shape)
df_methods.head()

In [ ]:
cols2join_methods = cols_methods[0:-1]
df_methods_multiindex = df_methods.set_index(cols2join_methods)

In [ ]:
df_methods_multiindex.head()

## DataFrame als Excel-Datei speichern

Der Excel-Datei werden alle notwendigen Metadaten hinzugefügt, damit sie ohne manuelle Anpassungen in das e1ns-Formblatt **PLATO e1ns - Formblatt Lizenzen** importiert werden kann.

*Für e1ns 2.1.3 ist noch ein Workaround notwendig, damit die Spalte "ist Katalog?" als BooleanType importiert wird.*

In [ ]:
# Workaround für einen Bug beim Import von Boolean:
# e1ns kann keine Boolean-Werte von Excel importieren.
# Es muss deshalb True und False als String in die Tabelle geschrieben werden.
df_kunden_multiindex_fix = df_kunden_multiindex.copy()
df_kunden_multiindex_fix['ist Katalog?'] = df_kunden_multiindex_fix['ist Katalog?'].apply(
    lambda val: 'True' if val else 'False'
)

writer = pd.ExcelWriter(r'C:\Users\entrup\Documents\Jupyter-Output\LIC-Export_full.xlsx', engine='xlsxwriter')
df_kunden_multiindex_fix.to_excel(writer, sheet_name='LIC-Full')
workbook  = writer.book

meta_sheet = workbook.add_worksheet('ImportDefinition')
col_ids = ['customer', 'file', 'signed', 'expires', 'method', 'ws_slots', 'user_slots', 'worksheet', 'is_catalog']
for col in range(len(cols_kunden)):
    meta_sheet.write(0, col, cols_kunden[col])
    meta_sheet.write(1, col, col_ids[col])
workbook.define_name('column_ids', '=ImportDefinition!A2:I2')
meta_sheet.write('A10', 'services_e1ns_ws_licences')
workbook.define_name('worksheet_definition', '=ImportDefinition!A10')
meta_sheet.write('A12', 'LIC-Full')
workbook.define_name('data_sheets', '=ImportDefinition!A12')
writer.save()

### Eine Excel-Datei mit einem Formblatt pro Kunden erstellen

Diese Excel-Datei wird nicht für den Import in e1ns genutzt.

In [ ]:
writer = pd.ExcelWriter(r'C:\Users\entrup\Documents\Jupyter-Output\LIC-Export_by_customer.xlsx', engine='xlsxwriter')
for kunde in sorted(kunden_licfiles.keys()):
    df_kunde = df_kunden_multiindex.loc[kunde]
    # Die Titel eines worksheets dürfen bei Excel nur 31 Zeichen besitzen:
    if len(kunde) > 31:
        kunde = kunde[:28] + '...'
    # Nicht unterstützte Sonderzeichen entfernen:
    kunde = kunde.replace('/', ' ')
    df_kunde.to_excel(writer, sheet_name=kunde)
writer.save()

### Verfügbare Methoden als Excel-Datei exportieren

In [ ]:
df_methods_multiindex_fix = df_methods_multiindex.copy()
df_methods_multiindex_fix['ist Katalog?'] = df_methods_multiindex_fix['ist Katalog?'].apply(
    lambda val: 'True' if val else 'False'
)

writer = pd.ExcelWriter(r'C:\Users\entrup\Documents\Jupyter-Output\Methods-Export.xlsx', engine='xlsxwriter')
df_methods_multiindex_fix.to_excel(writer, sheet_name='Methods')
workbook  = writer.book

meta_sheet = workbook.add_worksheet('ImportDefinition')
col_ids = ['method', 'ws_count', 'cat_count', 'worksheet', 'is_catalog']
for col in range(len(cols_methods)):
    meta_sheet.write(0, col, cols_methods[col])
    meta_sheet.write(1, col, col_ids[col])
workbook.define_name('column_ids', '=ImportDefinition!A2:I2')
meta_sheet.write('A10', 'lu_services_e1ns_methods')
workbook.define_name('worksheet_definition', '=ImportDefinition!A10')
meta_sheet.write('A12', 'Methods')
workbook.define_name('data_sheets', '=ImportDefinition!A12')
writer.save()